In [1]:
import numpy as np
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

embd = np.load('/content/drive/MyDrive/CS_768/saved_embeddings_alpha0.5beta0.5/Z_50_Epoch_1389_Loss_5006.720362227084.npy')


path_to_save = '/content/drive/MyDrive/CS_768/node_500/'
C_test = np.load(path_to_save + 'C_test.npy',allow_pickle=True)
S = np.load(path_to_save + 'S.npy',allow_pickle=True)
nodes = [i for i in range(len(S))]

In [1]:
Z1 = np.load('./saved_embeddings_alpha0.2beta0.8/Z_50_Epoch_131_Loss_4887.216622925517.npy')
Z2 = np.load('./Z_50_Epoch_975_Loss_5024.69774230853.npy')
Z3 = np.load('./saved_embeddings_alpha0.1beta0.9/Z_50_Epoch_440_Loss_4298.954994396465.npy')
path_to_save = './node_500/'
C_test = np.load(path_to_save + 'C_test.npy')
S = np.load(path_to_save + 'S.npy')
nodes = [i for i in range(len(S))]


In [6]:
def prob1(u,v, embd):
    b=(np.linalg.norm(embd[v]-embd[u]))**2
    a=1/(1+np.exp(b))
    return a

def prob_final(u,c, embd):
    sum=0;
    for i in range(math.ceil(0.1*c.shape[0])):
        sum=sum+math.log(1-prob1(u,c[i,0],embd))
    return(sum)

def calc_score(nodes,c, embd):
    r_hat=[]
    noninfected_nodes=list(set(nodes)-set(c[:math.ceil(0.1*c.shape[0]),0]))
    for i in range(len(noninfected_nodes)):
        r_hat.append([noninfected_nodes[i],prob_final(noninfected_nodes[i],c, embd)])
    r_hat.sort(key=lambda x: (x[1]))
    return r_hat
    
def precision(r_hat,r):
    r_hat_set=set(r_hat)
    r_set=set(r)
    return (len(r_hat_set.intersection(r_set))/(len(r_hat_set)+1e-5))

def average_precison(nodes,c,k, embd):
    r_hat=calc_score(nodes,c, embd)
    prcsn=0;
    m=0;
    s=set(c[math.ceil(0.1*c.shape[0]):math.ceil(0.1*c.shape[0])+k,0])
    for j in range(k):
        if(np.array(r_hat)[j,0] in s):
            m+=1;
            prcsn=prcsn+precision(np.array(r_hat)[0:j,0],c[0:,0])
    return (prcsn/m)

def mean_average_precision(nodes,C, embd):
    k=[50, 70, 100,120]
    mean_pre=[]
    for i in range(len(k)):
        mean_prcsn=0
        for j in range(len(C)):
            mean_prcsn += average_precison(nodes,C[j],k[i], embd)
        mean_pre.append(mean_prcsn/len(C))
    return mean_pre
        
